# 🔊 AI Generated Ad Films for Product Videography

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Wellsaid_Voiceover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

Crafting AI content and blending results from multiple tools can be incredibly powerful and time-efficient. Manual execution of these tasks is often cumbersome and time-consuming. However, with the right tools and techniques, you can automate various processes to achieve remarkable results effortlessly.

In this tutorial, we will explore how to leverage the seamless integration between `Wellsaid`, `OpenAI` and `VideoDB` to create captivating voiceovers for product advertisements. By harnessing the advanced capabilities of these platforms, you can enhance the storytelling and engagement of your product videos with AI-generated voiceovers; all within a single environment. 

For this tutorial, we’ll be using a product footage from Youtube 👉🏼 https://www.youtube.com/watch?v=2DcAMbmmYNM and convert it into a professionally made advertisement for a jewellery brand; all using AI and programmatic scripting & editing.

![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Wellsaid_Voiceover_1.png)

## Setup

---

### 📦  Installing packages 

Ensure you have the necessary packages installed:

In [ ]:
%pip install openai
%pip install videodb

### 🔑 API Keys

Ensure you have access to [OpenAI](https://openai.com), [Wellsaid](https://wellsaidlabs.com/features/api/) and [VideoDB](https://videodb.io) API keys. If you haven't already, sign up for API access on the respective platforms.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["WELLSAID_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

### 🎙️  Wellsaid's Voice Avatar 


You will also need Wellsaid's Voice Avatar that you want to use to generate your voiceover. 

For this demo, we’ll choose a slick, professional ads voiceover artist’s voice. Wellsaid has a wide range of options to choose from- you can check them out by signing up on their platform and accessing their [Voice Studio](https://docs.wellsaidlabs.com/reference/available-voice-avatars). Once finalized, copy the Speaker ID from Wellsaid and link it here.

In [3]:
speaker_id = "109"

## Implementation

---


### 🌐 Step 1: Connect to VideoDB
Begin by establishing a connection to VideoDB using your API key

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Video

Upload your product video to VideoDB using its URL or file path.

In [5]:
# Upload a video by URL (replace the url with your video)
video = conn.upload(url='https://www.youtube.com/watch?v=2DcAMbmmYNM')

### 🔍 Step 3: Analyze Scenes and Generate Scene Descriptions

Start by analyzing the scenes within your Video using VideoDB's scene indexing capabilities. This will provide context for the script prompt.

In [6]:
video.index_scenes()

Let's view the description of first scene of the video

In [7]:
scenes = video.get_scenes()
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

0 - 0.5338666666666667
The image presented is overwhelmingly dark, with different shades of black and deep blues blending into each other. There are no discernible objects, figures, or distinct shapes to provide context or detail. The top right corner holds a small watermark or icon with illegible text, suggesting some form of branding or ownership. This image conveys a sense of emptiness or perhaps an abstract concept, as it lacks visual content that would normally inform the viewer's interpretation. It could represent a darkroom, outer space, or even an image error. The nature of this image is enigmatic and open-ended, allowing for various interpretations.


### Step 4: Generate Voiceover Script with LLM
Combine scene descriptions with a script prompt, instructing LLM (OpenAI) to create a suitable voiceover for your product advertisement.

This script prompt can be refined and tweaked to generate the most suitable output. Check out [these examples](https://www.youtube.com/playlist?list=PLhxAMFLSSK03rsPTjRv1LbAXHQpNN6BS0) to explore more use cases.

In [8]:
import openai

client = openai.OpenAI()

script_prompt = "Here's the data from a scene index for a video from a jewellery product photoshoot. Study this and then generate a synced script based on the description below. Make sure the script is in the language, voice and style of a professional voiceover artist skilled at weaving beautiful storytelling in advertisements.\n \n"

full_prompt = script_prompt + "\n\n"
for scene in scenes:
  full_prompt += f"- {scene}\n"

openai_res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": full_prompt}],
)
voiceover_script = openai_res.choices[0].message.content


# Truncate first 1000 characters of script
# If you have Wellsaid's paid plan remove this
voiceover_script = voiceover_script[:1000]

### 🎤 Step 5: Generate Voiceover Audio with Wellsaid 

Utilize the generated script to synthesize a professional voiceover for your product advertisement using Wellsaid's API.

In [9]:
import requests
import time 


# Call Wellsaid API to generate voiceover
url = "https://api.wellsaidlabs.com/v1/tts/stream"
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "X-Api-Key": os.environ.get("WELLSAID_API_KEY")
}

# Initiate TTS Job for each Chunk
payload = {
    "text": voiceover_script,
    "speaker_id": speaker_id
}
wellsaid_res = requests.request("POST", url, json=payload, headers=headers)

# Save the audio file
audio_file = "audio.mp3"
CHUNK_SIZE = 1024
with open(audio_file, 'wb') as f:
    for chunk in wellsaid_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

### 🎬 Step 6: Add Voiceover to Video with VideoDB

In order to use the voiceover generated above, let's upload the audio file (voiceover) to VideoDB first

In [10]:
audio = conn.upload(file_path=audio_file)

Finally, add the AI-generated voiceover to the original footage using VideoDB's [timeline feature](https://docs.videodb.io/version-0-0-3-timeline-and-assets-44)

In [11]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

# Create a timeline object
timeline = Timeline(conn)

# Add the video asset to the timeline for playback
video_asset = VideoAsset(asset_id=video.id)
timeline.add_inline(asset=video_asset)

# Add the audio asset to the timeline for playback
audio_asset = AudioAsset(asset_id=audio.id)
timeline.add_overlay(start=0, asset=audio_asset)

### 🪄 Step 7: Review and Share

Preview the product video with the integrated voiceover to ensure it aligns with your vision. Once satisfied, share the video to showcase your product effectively.


In [1]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)

---

### 🎉 Conclusion:
Congratulations! You have successfully automated the process of creating AI-generated voiceovers for product videography using Wellsaid and VideoDB.

Experiment with different script prompts and scene analysis techniques to tailor the voiceovers to your specific product and audience. We look forward to seeing your captivating product videos!

For more such explorations, refer to the [documentation of VideoDB](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.

We're excited to see your creations, so we welcome you to share your creations via [Discord](https://discord.com/invite/py9P639jGz), [LinkedIn](https://www.linkedin.com/company/videodb) or [Twitter](https://twitter.com/videodb_io).